In [12]:
train_text, train_target = [],[]

In [15]:
with open('data/train.txt', 'r') as f:
    for line in f:
        parts = line.strip().split(';')
        if len(parts) != 2:
            continue
        train_text.append(parts[0])
        train_target.append(parts[1])
    f.close()

In [38]:
valid_text, valid_target = [],[]

In [39]:
with open('data/val.txt', 'r') as f:
    for line in f:
        parts = line.strip().split(';')
        if len(parts) != 2:
            continue
        valid_text.append(parts[0])
        valid_target.append(parts[1])
    f.close()

In [17]:
train_text, train_target

(['i didnt feel humiliated',
  'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
  'im grabbing a minute to post i feel greedy wrong',
  'i am ever feeling nostalgic about the fireplace i will know that it is still on the property',
  'i am feeling grouchy',
  'ive been feeling a little burdened lately wasnt sure why that was',
  'ive been taking or milligrams or times recommended amount and ive fallen asleep a lot faster but i also feel like so funny',
  'i feel as confused about life as a teenager or as jaded as a year old man',
  'i have been with petronas for years i feel that petronas has performed well and made a huge profit',
  'i feel romantic too',
  'i feel like i have to make the suffering i m seeing mean something',
  'i do feel that running is a divine experience and that i can expect to have some type of spiritual encounter',
  'i think it s the easiest time of year to feel dissatisfied',
  'i feel low energy i

In [18]:
import re

In [19]:
def remove_url(text):
    return re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

In [20]:
def lowercase(text):
    return text.lower()

In [22]:
pip install nltk

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached regex-2024.11.6-cp312-cp312-win_amd64.whl.metadata (41 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached regex-2024.11.6-cp312-cp312-win_amd64.whl (273 kB)
Using cached click-8.2.1-py3-none-any.whl (102 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
!pip install contractions


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:
import contractions

In [33]:
def standardize_text(text):
    text = remove_url(text)
    text = lowercase(text)
    return contractions.fix(text)


In [36]:
def delete_stopwords(text):
    stop_words = set(stopwords.words('english'))
    texts = standardize_text(text)
    tokens = word_tokenize(texts)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)

In [34]:
text = train_text[0]
text

'i didnt feel humiliated'

In [35]:
standardize_text(text)

'i did not feel humiliated'

In [37]:
delete_stopwords(text)

'feel humiliated'

In [44]:
from nltk.corpus import wordnet
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [59]:
def get_tranform(tag):
  if tag.startswith('J'):
    return wordnet.ADJ
  elif tag.startswith('V'):
    return wordnet.VERB
  elif tag.startswith('N'):
    return wordnet.NOUN
  elif tag.startswith('R'):
    return wordnet.ADV
  else:
    return wordnet.NOUN

In [69]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    tokens = word_tokenize(text)
    words_and_tags = nltk.pos_tag(tokens)
    lemmatized_words = [lemmatizer.lemmatize(word, pos=get_tranform(tag)) for word, tag in words_and_tags]
    return ' '.join(lemmatized_words)


In [40]:
train_text = [delete_stopwords(text) for text in train_text]
valid_text = [delete_stopwords(text) for text in valid_text]

In [47]:
text = train_text[0]


In [70]:
lemmatize_text(text)

'feel humiliate'

In [42]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [71]:
train_text = [lemmatize_text(text) for text in train_text]
valid_text = [lemmatize_text(text) for text in valid_text]

In [84]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier

In [ ]:
vocab = set()
for text in train_text:
    tokens = word_tokenize(text)
    vocab.update(tokens)

In [78]:
pipe = Pipeline([
    ('vect', CountVectorizer(vocabulary=vocab)),
    ('tfidf', TfidfTransformer())
])
pipe.fit(train_text)
train_text = pipe.transform(train_text)
valid_text = pipe.transform(valid_text)

In [79]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_target = le.fit_transform(train_target)
valid_target = le.transform(valid_target)

In [85]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
lib = {
    'LogisticRegression': LogisticRegression(),
    'DecisionTreeClassifier': DecisionTreeClassifier(),
    'RandomForestClassifier': RandomForestClassifier(),
    'MLPClassifier': MLPClassifier()
}
for name, model in lib.items():
    model.fit(train_text, train_target)
    pred = model.predict(valid_text)
    print(f"Model: {name}")
    print(classification_report(valid_target, pred, target_names=le.classes_))
    print(confusion_matrix(valid_target, pred))
    print("\n")

Model: LogisticRegression
              precision    recall  f1-score   support

       anger       0.88      0.82      0.85       275
        fear       0.84      0.75      0.79       212
         joy       0.85      0.94      0.89       704
        love       0.85      0.61      0.71       178
     sadness       0.87      0.93      0.90       550
    surprise       0.88      0.54      0.67        81

    accuracy                           0.86      2000
   macro avg       0.86      0.77      0.80      2000
weighted avg       0.86      0.86      0.85      2000

[[225   6  21   1  22   0]
 [  9 160  18   1  21   3]
 [  6   2 663  15  16   2]
 [  3   3  52 108  12   0]
 [  9   8  17   2 513   1]
 [  3  12  13   0   9  44]]


Model: DecisionTreeClassifier
              precision    recall  f1-score   support

       anger       0.79      0.81      0.80       275
        fear       0.77      0.83      0.80       212
         joy       0.86      0.84      0.85       704
        love       